# Perturbed embeddings
## Novel approaches in data augmentation for low-resource machine translation

Andrew Yang

In [1]:
from load_util import *

import numpy as np
import tensorflow.keras as K
from nltk.translate.bleu_score import corpus_bleu

In [2]:
MODEL_FILEPATH = 'models/full-data-model.h5'
EMBED_DIM = 300
LSTM1_SIZE = 256
LSTM2_SIZE = 256

In [3]:
data = preprocess(load_data('deu-eng/deu.txt'))
data.shape

(204574, 2)

In [4]:
data = data[:30000]
np.random.seed(0)
np.random.shuffle(data)
SRC, TGT = data[:, 0], data[:, 1]
SRC_enc_pad, SRC_tokenizer = encode_and_pad(SRC)
TGT_enc_pad, TGT_tokenizer = encode_and_pad(TGT)
SRC_vocab_len = vocab_len(SRC_enc_pad)
TGT_vocab_len = vocab_len(TGT_enc_pad)
SRC_train, SRC_test, TGT_train, TGT_test = train_split(SRC_enc_pad, TGT_enc_pad, train_size=.9)

In [5]:
TGT_train_onehot = onehot_3d(TGT_train, TGT_vocab_len)
TGT_test_onehot = onehot_3d(TGT_test, TGT_vocab_len)

In [6]:
def build_model(
    input_vocab_len,
    input_max_len,
    output_vocab_len,
    output_max_len,
    embed_dim,
    lstm1_units,
    lstm2_units):
    
    model = K.models.Sequential()
    model.add(K.layers.Embedding(
        input_dim=input_vocab_len,
        output_dim=embed_dim,
        mask_zero=True,
        input_length=input_max_len))
    model.add(K.layers.LSTM(units=lstm1_units))
    model.add(K.layers.RepeatVector(n=output_max_len))
    model.add(K.layers.LSTM(units=lstm2_units, return_sequences=True))
    model.add(K.layers.TimeDistributed(K.layers.Dense(output_vocab_len, activation='softmax')))
    
    return model

In [7]:
model = build_model(
    input_vocab_len=SRC_vocab_len,
    input_max_len=SRC_train.shape[1],
    output_vocab_len=TGT_vocab_len,
    output_max_len=TGT_train_onehot.shape[1],
    embed_dim=EMBED_DIM,
    lstm1_units=LSTM1_SIZE,
    lstm2_units=LSTM2_SIZE
)
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 6, 300)            1407900   
_________________________________________________________________
lstm (LSTM)                  (None, 256)               570368    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 10, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 256)           525312    
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 7432)          1910024   
Total params: 4,413,604
Trainable params: 4,413,604
Non-trainable params: 0
_________________________________________________________________


In [8]:
model_checkpoint = K.callbacks.ModelCheckpoint(filepath=MODEL_FILEPATH, monitor='val_loss', verbose=True, save_best_only=True)
model.fit(
    x=SRC_train,
    y=TGT_train_onehot,
    batch_size=64,
    epochs=30,
    verbose=True,
    callbacks=[model_checkpoint],
    validation_data=[SRC_test, TGT_test_onehot])

Train on 27000 samples, validate on 3000 samples
Epoch 1/40
26944/27000 [============================>.] - ETA: 0s - loss: 2.8724
Epoch 00001: val_loss improved from inf to 2.43471, saving model to models/full-data-model.h5
27000/27000 [==============================] - 32s 1ms/sample - loss: 2.8715 - val_loss: 2.4347
Epoch 2/40
26944/27000 [============================>.] - ETA: 0s - loss: 2.3369
Epoch 00002: val_loss improved from 2.43471 to 2.27912, saving model to models/full-data-model.h5
27000/27000 [==============================] - 13s 494us/sample - loss: 2.3369 - val_loss: 2.2791
Epoch 3/40
26944/27000 [============================>.] - ETA: 0s - loss: 2.1541
Epoch 00003: val_loss improved from 2.27912 to 2.13746, saving model to models/full-data-model.h5
27000/27000 [==============================] - 13s 491us/sample - loss: 2.1537 - val_loss: 2.1375
Epoch 4/40
26944/27000 [============================>.] - ETA: 0s - loss: 1.9916
Epoch 00004: val_loss improved from 2.13746 t

KeyboardInterrupt: 

In [9]:
trained_model = K.models.load_model(MODEL_FILEPATH)

In [10]:
evaluate(model=trained_model, X=SRC_train, Y=TGT_train, X_tokenizer=SRC_tokenizer, Y_tokenizer=TGT_tokenizer)

halten sie mir einen platz frei  -->  halt mir einen platz frei
ich muss ihn warnen  -->  ich muss ihn warnen
tom war stark  -->  tom war stark
lassen sie mich das erledigen  -->  lass mich das das
sie belieben wohl zu scherzen  -->  du beliebt wohl zu scherzen
verpetzen sie mich nicht  -->  verpetz mich nicht
verpiss dich  -->  mach dich
tom ist wieder da  -->  tom ist wieder
ich bin arzt  -->  ich bin arzt
du bist krank  -->  du bist krank
tom spielt rugby  -->  tom spielt kricket
wo sind die schlssel  -->  wo sind die schlssel
es ist jetzt da  -->  es ist jetzt da
tom war ein starker trinker  -->  tom war ein starker
sie faszinierte mich  -->  sie faszinierte mich
tom ist leichtglubig  -->  tom ist leichtglubig
er ist ein schneller lufer  -->  er rennt schnell
ihr habt pech  -->  du hast pech
ich werde dich anrufen  -->  ich werde dich anrufen
alle wissen es  -->  alle wissen es
1-gram BLEU: 0.7737
2-gram BLEU: 0.6900
3-gram BLEU: 0.6120
4-gram BLEU: 0.4914


In [11]:
evaluate(model=trained_model, X=SRC_test, Y=TGT_test, X_tokenizer=SRC_tokenizer, Y_tokenizer=TGT_tokenizer)

vgel singen  -->  vgel
ich kannte tom nicht  -->  ich habe tom nicht gekannt
wer ist eure freundin  -->  wer ist euer freundin
tom wrde es wissen  -->  tom wrde es
tom warf den ball  -->  tom hat den ball
beruhigen sie sich  -->  beruhigen sie
ich helfe  -->  ich werde helfen
kann ich ihnen helfen  -->  darf ich dir helfen
tom bat um hilfe  -->  tom hat mit nicht
lass uns diesen kaufen  -->  lasst uns uns kaufen
tom hat veranlasst dass maria geht  -->  tom hat maria tom zu gebracht
dies ist nicht mein haus  -->  das ist nicht nicht tom
tom hat gebeichtet  -->  tom hat kaninchen
maria singt alt  -->  sie singt sopran
tom wird heulen  -->  tom wird weinen
er hat kein fahrrad  -->  er hat keinen ein
du liest meine gedanken  -->  du brauchen mir ein
ich bin eine optimistin  -->  ich bin ein optimist
tom ist gefunden worden  -->  tom ist gefunden
ich mag den hund  -->  ich mag die hund
1-gram BLEU: 0.5832
2-gram BLEU: 0.4602
3-gram BLEU: 0.3554
4-gram BLEU: 0.2478
